In [1]:
import json
import pandas as pd

from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

In [2]:
with open("../.oauth", "r") as fobj:
     cid, secret = [s.rstrip('\n') for s in fobj.readlines()]

In [3]:
%%capture output
%%bash -s "$cid" "$secret"
curl -u $1:$2 -d grant_type=client_credentials https://classic.warcraftlogs.com/oauth/token

In [4]:
token = json.loads(output.stdout)['access_token']

transport = AIOHTTPTransport(
    url='https://classic.warcraftlogs.com/api/v2/client', 
    headers={'Authorization': 'Bearer ' + token}, 
    timeout=120
)
client = Client(transport=transport, fetch_schema_from_transport=True)

In [9]:
zone_id = 1007
encounter_id = 652
start_time = 0
limit = 10
pages = 1
page = 1

data = {}

query = gql("""
    query ($zoneID: Int!, $encounterID: Int!, $startTime: Float!, $limit: Int!, $page: Int!) {
      reportData {
        reports(zoneID: $zoneID, guildServerRegion: "US", startTime: $startTime, limit: $limit, page: $page) {
          data {
            events(encounterID: $encounterID, startTime: 0, endTime: 999999999, limit: 20000) {
               data
            }
          }
        }
      }
    }
""")

res = await client.execute_async(query, {
    "zoneID": zone_id,
    "encounterID": encounter_id,
    "startTime": start_time,
    "limit": limit,
    "page": page,
})

https://tbc.wowhead.com/spell=31687/summon-water-elemental

https://tbc.wowhead.com/spell=31707/waterbolt

In [48]:
data = res["reportData"]["reports"]["data"]

In [58]:
set([event["type"] for event in data[0]["events"]["data"]])

{'absorbed',
 'applybuff',
 'applybuffstack',
 'applydebuff',
 'applydebuffstack',
 'begincast',
 'cast',
 'combatantinfo',
 'damage',
 'death',
 'dispel',
 'encounterend',
 'encounterstart',
 'energize',
 'extraattacks',
 'heal',
 'refreshbuff',
 'refreshdebuff',
 'removebuff',
 'removebuffstack',
 'removedebuff',
 'removedebuffstack',
 'summon'}

In [104]:
water_infos = []
for i, report in enumerate(data):
    events = report["events"]["data"]
    for event in events:
        if event["type"] == "combatantinfo" and event["sourceID"] == 25:
            print(i, event)

1 {'timestamp': 571786, 'type': 'combatantinfo', 'sourceID': 25, 'gear': [{'id': 16939, 'quality': 4, 'icon': 'inv_helmet_05.jpg', 'itemLevel': 76, 'permanentEnchant': 2586}, {'id': 35507, 'quality': 4, 'icon': 'inv_jewelry_necklace_18.jpg', 'itemLevel': 110}, {'id': 27801, 'quality': 3, 'icon': 'inv_shoulder_23.jpg', 'itemLevel': 115, 'permanentEnchant': 2606, 'gems': [{'id': 24028, 'itemLevel': 70, 'icon': 'inv_jewelcrafting_livingruby_03.jpg'}, {'id': 24028, 'itemLevel': 70, 'icon': 'inv_jewelcrafting_livingruby_03.jpg'}]}, {'id': 0, 'quality': 1, 'icon': 'inv_axe_02.jpg', 'itemLevel': 0}, {'id': 28186, 'quality': 3, 'icon': 'inv_chest_leather_04.jpg', 'itemLevel': 112, 'gems': [{'id': 28595, 'itemLevel': 60, 'icon': 'inv_misc_gem_bloodgem_02.jpg'}, {'id': 28595, 'itemLevel': 60, 'icon': 'inv_misc_gem_bloodgem_02.jpg'}, {'id': 28595, 'itemLevel': 60, 'icon': 'inv_misc_gem_bloodgem_02.jpg'}]}, {'id': 31293, 'quality': 3, 'icon': 'inv_belt_31.jpg', 'itemLevel': 112}, {'id': 27874, 'qu

In [69]:
water_summs = []
for i, report in enumerate(data):
    events = report["events"]["data"]
    for event in events:
        if event["type"] == "summon" and event["abilityGameID"] == 31687:
            water_summs.append((i, event))

In [70]:
water_summs

[(4,
  {'timestamp': 589821,
   'type': 'summon',
   'sourceID': 11,
   'targetID': 25,
   'targetInstance': 1,
   'abilityGameID': 31687}),
 (4,
  {'timestamp': 695918,
   'type': 'summon',
   'sourceID': 11,
   'targetID': 25,
   'targetInstance': 2,
   'abilityGameID': 31687}),
 (6,
  {'timestamp': 461893,
   'type': 'summon',
   'sourceID': 5,
   'targetID': 26,
   'abilityGameID': 31687}),
 (6,
  {'timestamp': 462404,
   'type': 'summon',
   'sourceID': 13,
   'targetID': 37,
   'abilityGameID': 31687})]

In [80]:
# Looking at a deep frostie boi
report_4 = data[4]["events"]["data"]
for event in report_4:
    if event["type"] == "combatantinfo" and event["sourceID"] == 11:
        frost_mage = event

In [83]:
pd.json_normalize(frost_mage, sep="_").iloc[0]

timestamp                                                             581108
type                                                           combatantinfo
sourceID                                                                  11
gear                       [{'id': 29076, 'quality': 4, 'icon': 'inv_helm...
auras                      [{'source': 10, 'ability': 27127, 'stacks': 1,...
expansion                                                                tbc
faction                                                                    0
specID                                                                     0
strength                                                                  46
agility                                                                   51
stamina                                                                  549
intellect                                                                505
spirit                                                                   220

Nice, we can get hit, crit and int. SP doesn't seem readily computable.

In [86]:
frost_mage["gear"]

[{'id': 29076,
  'quality': 4,
  'icon': 'inv_helmet_84.jpg',
  'itemLevel': 120,
  'permanentEnchant': 3002,
  'gems': [{'id': 34220,
    'itemLevel': 70,
    'icon': 'inv_misc_gem_diamond_07.jpg'},
   {'id': 24056,
    'itemLevel': 70,
    'icon': 'inv_jewelcrafting_nightseye_03.jpg'}]},
 {'id': 28134,
  'quality': 3,
  'icon': 'inv_jewelry_necklace_27.jpg',
  'itemLevel': 112},
 {'id': 27796,
  'quality': 3,
  'icon': 'inv_shoulder_22.jpg',
  'itemLevel': 115,
  'permanentEnchant': 2994,
  'gems': [{'id': 30608,
    'itemLevel': 70,
    'icon': 'inv_jewelcrafting_talasite_03.jpg'},
   {'id': 30605,
    'itemLevel': 70,
    'icon': 'inv_jewelcrafting_talasite_03.jpg'}]},
 {'id': 6096, 'quality': 2, 'icon': 'inv_shirt_01.jpg', 'itemLevel': 1},
 {'id': 27799,
  'quality': 3,
  'icon': 'inv_chest_cloth_18.jpg',
  'itemLevel': 115,
  'permanentEnchant': 3150},
 {'id': 28654, 'quality': 4, 'icon': 'inv_belt_03.jpg', 'itemLevel': 115},
 {'id': 27907,
  'quality': 3,
  'icon': 'inv_pants_cl

In [66]:
waterbolts = []
for report in data:
    events = report["events"]["data"]
    for event in events:
        if event["type"] == "damage" and event["abilityGameID"] == 31707:
            waterbolts.append(event)

In [67]:
waterbolts

[{'timestamp': 594283,
  'type': 'damage',
  'sourceID': 25,
  'sourceInstance': 1,
  'targetID': 32,
  'abilityGameID': 31707,
  'hitType': 1,
  'amount': 1051,
  'unmitigatedAmount': 910},
 {'timestamp': 597254,
  'type': 'damage',
  'sourceID': 25,
  'sourceInstance': 1,
  'targetID': 32,
  'abilityGameID': 31707,
  'hitType': 1,
  'amount': 1056,
  'unmitigatedAmount': 914},
 {'timestamp': 600440,
  'type': 'damage',
  'sourceID': 25,
  'sourceInstance': 1,
  'targetID': 32,
  'abilityGameID': 31707,
  'hitType': 1,
  'amount': 997,
  'unmitigatedAmount': 863},
 {'timestamp': 603756,
  'type': 'damage',
  'sourceID': 25,
  'sourceInstance': 1,
  'targetID': 32,
  'abilityGameID': 31707,
  'hitType': 1,
  'amount': 991,
  'unmitigatedAmount': 858},
 {'timestamp': 606517,
  'type': 'damage',
  'sourceID': 25,
  'sourceInstance': 1,
  'targetID': 32,
  'abilityGameID': 31707,
  'hitType': 1,
  'amount': 1045,
  'unmitigatedAmount': 905},
 {'timestamp': 609965,
  'type': 'damage',
  's

Looks like the sourceID of the summon is the mage and the targetID is the water elemental - the waterbolts sourceIDs match to the summoned elementals. This actually looks really manageable - for each fight we can tag each water elemental to a mage, then we just need to determine if water elemental got drums / hero. We might be able to pick that explicit via an applybuff event, if not we'll just have to guess.

In [100]:
buffs = []
idx = [((25,), 4), ((26, 37), 6)]

for target_ids, index in idx:
    events = data[index]["events"]["data"]
    for event in events:
        if event["type"] in ["applybuff", "removebuff"] and event["targetID"] in target_ids:
            buffs.append(event)

In [101]:
buffs

[{'timestamp': 703650,
  'type': 'applybuff',
  'sourceID': 12,
  'targetID': 25,
  'targetInstance': 2,
  'abilityGameID': 35476},
 {'timestamp': 733651,
  'type': 'removebuff',
  'sourceID': 12,
  'targetID': 25,
  'targetInstance': 2,
  'abilityGameID': 35476},
 {'timestamp': 467037,
  'type': 'applybuff',
  'sourceID': 5,
  'targetID': 26,
  'abilityGameID': 28142},
 {'timestamp': 467037,
  'type': 'applybuff',
  'sourceID': 5,
  'targetID': 37,
  'abilityGameID': 28142},
 {'timestamp': 483789,
  'type': 'applybuff',
  'sourceID': 6,
  'targetID': 26,
  'abilityGameID': 2825},
 {'timestamp': 483789,
  'type': 'applybuff',
  'sourceID': 6,
  'targetID': 37,
  'abilityGameID': 2825},
 {'timestamp': 489795,
  'type': 'applybuff',
  'sourceID': 6,
  'targetID': 26,
  'abilityGameID': 35475},
 {'timestamp': 489795,
  'type': 'applybuff',
  'sourceID': 6,
  'targetID': 37,
  'abilityGameID': 35475},
 {'timestamp': 507056,
  'type': 'removebuff',
  'sourceID': 5,
  'targetID': 26,
  'abil

- https://tbc.wowhead.com/spell=35476/drums-of-battle
- https://www.wowhead.com/spell=28142/power-of-the-guardian

Nice - no https://tbc.wowhead.com/spell=32182/heroism but from this sample I can assume I can infer their buff by looking at the events exactly.


### TODO

- Create module for code reuse.
- Pull out client auth code into module.
- Create function for loop snippet above.
